# Terminal Processing

© Crown-owned copyright 2024, Defence Science and Technology Laboratory UK

This notebook serves as a guide on the functionality and use of the new Terminal simulation component.

By default, the Terminal will come pre-installed on any simulation component which inherits from `HostNode` (Computer, Server, Printer), and simulates the Secure Shell (SSH) protocol as the communication method.

In [ ]:
from primaite.simulator.system.services.terminal.terminal import Terminal
from primaite.simulator.network.container import Network
from primaite.simulator.network.hardware.nodes.host.computer import Computer

In [ ]:
def basic_network() -> Network:
    """Utility function for creating a default network to demonstrate Terminal functionality"""
    network = Network()
    node_a = Computer(hostname="node_a", ip_address="192.168.0.10", subnet_mask="255.255.255.0", start_up_duration=0)
    node_a.power_on()
    node_b = Computer(hostname="node_b", ip_address="192.168.0.11", subnet_mask="255.255.255.0", start_up_duration=0)
    node_b.power_on()
    network.connect(node_a.network_interface[1], node_b.network_interface[1])
    return network

demonstrate how we obtain the Terminal component

In [ ]:
network: Network = basic_network()
computer_a: Computer = network.get_node_by_hostname("node_a")
terminal_a: Terminal = computer_a.software_manager.software.get("Terminal")
computer_b: Computer = network.get_node_by_hostname("node_b")
terminal_b: Terminal = computer_b.software_manager.software.get("Terminal")

# Login to the remote (node_b) from local (node_a)
terminal_a.login(username="admin", password="Admin123!", ip_address=computer_b.network_interface[1].ip_address)


You can view all remote connections to a terminal through use of the `show()` method

In [ ]:
terminal_b.show()

The Terminal can be used to install new software. The code block below demonstrates how the Terminal class allows the user of `terminal_a`, on `computer_a`, to send a command to `computer_b` to install the `RansomwareScript` application. 


In [ ]:
from primaite.simulator.network.protocols.ssh import SSHConnectionMessage, SSHPacket, SSHTransportMessage
from primaite.simulator.system.applications.red_applications.ransomware_script import RansomwareScript

computer_b.software_manager.show()

payload: SSHPacket = SSHPacket(
    payload=["software_manager", "application", "install", "RansomwareScript"],
    transport_message=SSHTransportMessage.SSH_MSG_SERVICE_REQUEST,
    connection_message=SSHConnectionMessage.SSH_MSG_CHANNEL_OPEN,
    sender_ip_address=computer_a.network_interface[1].ip_address,
    target_ip_address=computer_b.network_interface[1].ip_address,
)

# Send commmand to install RansomwareScript
terminal_a.send(payload=payload, dest_ip_address=computer_b.network_interface[1].ip_address)

The `RansomwareScript` can then be seen in the list of applications on the `node_b Software Manager`.  

In [ ]:
computer_b.software_manager.show()

The below example shows how you can send a command via the terminal to create a folder on the target Node.

Here, we send a command to `computer_b` to create a new folder titled "Downloads".

In [ ]:
computer_b.file_system.show()

payload: SSHPacket = SSHPacket(
    payload=["file_system", "create", "folder", "Downloads"],
    transport_message=SSHTransportMessage.SSH_MSG_SERVICE_REQUEST,
    connection_message=SSHConnectionMessage.SSH_MSG_CHANNEL_OPEN,
    sender_ip_address=computer_a.network_interface[1].ip_address,
    target_ip_address=computer_b.network_interface[1].ip_address,
)

terminal_a.send(payload=payload, dest_ip_address=computer_b.network_interface[1].ip_address)

computer_b.file_system.show()

The resultant call to `computer_b.file_system.show()` shows that the new folder has been created.